- protection contre l'injection dans les args
- field validator pour personnalise les conditions d'acceptation (ex. avoir un champs positif si c'est un nombre, avoir un @ si on veut un email)
- sérialisation/désérialisation en JSON

# BaseModel

## Class BaseModel

In [ ]:
"""
Les attributs de classes à instancier n'ont pas besoin d'être passés dans le constructeur __init__
Chaque attribut de classe nécessite d'avoir son type déclaré.
Si un champs n'a pas de valeur assigné par défaut, il est mandatory 
Pour rendre un champs optionnel, il faut lui passer None
"""

from pydantic import BaseModel # no qa

class Hero(BaseModel):
    id: int
    name: str = "Thor" # When a default value is set, this field is Optionnal

# Only giving mandatory field id
my_hero = Hero(
    id=0,
    )
print(my_hero)

# Adding an unknown field
your_hero = Hero(
    id=1,
    name="The Burden",
    age=25, # age is unknown but it won't raise an error, just ignored
    )
print(your_hero)

id=0 name='Thor'
id=1 name='The Burden'


## Type Coercion

In [21]:
from datetime import datetime

from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: datetime | None = None
    friends: list[int] = []


external_data = {
    "id": "123",
    "signup_ts": "2017-06-01 12:22",
    "friends": [1, "2", b"3"],
}
user = User(**external_data)
print(user)
# > User id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
print(user.id, type(user.id))
# > 123
print(user.signup_ts, type(user.signup_ts))

id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
123 <class 'int'>
2017-06-01 12:22:00 <class 'datetime.datetime'>


## Fonction pydantic.create_model()

In [ ]:
"""
La fonction create_model est particulièrement utile lorsque vous avez besoin de générer des modèles basés
sur des informations disponibles uniquement au moment de l’exécution.
Cela permet de rendre votre code plus flexible et adaptable aux différentes situations.
"""

from pydantic import create_model # noqa

# Création d'un modèle dynamique
PersonModel = create_model(
    'Person', # Nom de la classe
    name=(str, ...), # Attribut de classe > nom = Tuple(type, None|...) None = Optionnel, ... = Obligatoire
    age=(int, ...), # Cela revient à déclarer age: int
    email=(str, None) # Cela revient à déclarer email: str = None
    )

# Utilisation du modèle
person = PersonModel(name='John Doe', age=30, email='john@example.com')
print(person)


name='John Doe' age=30 email='john@example.com'


## Serialisation / Deserialisation

In [31]:
from pydantic import BaseModel # no qa

class Hero(BaseModel):
    id: int
    name: str = "Thor" # When a default value is set, this field is Optionnal

# Only giving mandatory field id
my_hero = Hero(
    id=0,
    )
print(my_hero)

# Sérialisation en JSON
hero_json = my_hero.model_dump()
print(hero_json)  # Affiche: {"id": 0, "name": "Thor"}

# Désérialisation à partir de JSON
new_hero = Hero.model_validate(hero_json)
print(new_hero)  # Affiche: id=0 name='Thor'


id=0 name='Thor'
{'id': 0, 'name': 'Thor'}
id=0 name='Thor'


# ModelConfig

## Class ConfigDict

### Set up a parent class only for config
    

In [ ]:
from pydantic import BaseModel, ConfigDict


class Parent(BaseModel):
    model_config = ConfigDict(extra='allow')


class Model(Parent):
    x: str


m = Model(x='foo', y='bar')
print(m.model_dump())
#> {'x': 'foo', 'y': 'bar'}

{'x': 'foo', 'y': 'bar'}


### Attributs : str_max_length, extra

In [ ]:
"""
Renseigner l'attribut de classe model_config inhérité de BaseModel en utilisant pydantic.ConfigDict.
str_max_length : longueur maximale de valeur du champs
extra : 'ignore', 'allow', ou 'forbid' > gestion des attributs non définis dans le modèle
"""

from pydantic import BaseModel, ConfigDict, ValidationError # noqa

class Hero(BaseModel):
    # Attributes usually defined in the __init__ constructor
    id: int
    name: str = "Thor"

    # model_config is an attribute inherited from BaseModel class
    model_config = ConfigDict(
        str_max_length=10,
        extra='forbid' # Forbid will raise an ValidationError if you instantiate a class object with unknown attribute
        )

# Testing a name qui a plus de 10 caractères
try:
    Hero(id=2, name="Nice Looking Minotausorus")
except ValidationError as error:
    print(error)

1 validation error for Hero
name
  String should have at most 10 characters [type=string_too_long, input_value='Nice Looking Minotausorus', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_long


### Attributs: ignore_types

In [29]:
"""
Permets d'ignorer des champs que vous ne voulez pas inclure dans la validation ou la sérialisation.
Raise a PydanticUserError
Les propriétés (attributs découlant d'une méthode utilisé avec un décorateur @property ou @cached_property ne sont pas soumis à validation et sérialisation)
Fonctionne avec des descripteurs personnalisés tel que CachedProperty de asyncstdlib
"""

from pydantic import BaseModel, ConfigDict, PydanticUserError # noqa
# from functools import cached_property # noqa
from asyncstdlib.functools import CachedProperty # noqa

class MyModel(BaseModel):
    model_config = ConfigDict(ignored_types=(CachedProperty,))
    id: int = 1

    @CachedProperty
    def cached_value(self):
        return 'cached'

model = MyModel()
print(model.cached_value)  # Affiche 'cached' sans soulever d'erreur
print(model.model_dump()) # Sérialise l'objet instancié sans la cached property

try:
    class MyOtherModel(BaseModel):
        model_config = ConfigDict(ignored_types=())
        id: int = 1

        @CachedProperty
        def cached_value(self):
            return 'cached'
except PydanticUserError as e:
    print(e)


<_FutureCachedPropertyValue for 'MyModel.cached_value' at 0x773c572e4c00>
{'id': 1}
A non-annotated attribute was detected: `cached_value = <asyncstdlib.functools.CachedProperty object at 0x773c6c35cd40>`. All model fields require a type annotation; if `cached_value` is not meant to be a field, you may be able to resolve this error by annotating it as a `ClassVar` or updating `model_config['ignored_types']`.

For further information visit https://errors.pydantic.dev/2.10/u/model-field-missing-annotation


### Attribut: alias_generator

In [ ]:
"""
Permets de pouvoir utiliser une clé orthographié ou avec une casse différente lors de l'instantiation d'un objet.
"""

from pydantic import BaseModel, ConfigDict # noqa

def temporary_alias(field_name: str) -> str:
    return field_name.upper()

class MyModel(BaseModel):
    model_config = ConfigDict(alias_generator=temporary_alias)
    my_field: str # field name in lowercase

# Utilisation
data = {'MY_FIELD': 'value'} # Mfield name in uppercase is valid with the alias_generator
model = MyModel(**data)
print(model.my_field)  # Affiche 'value'

value


## BaseModel KWArgs

In [ ]:
# WAY 2: The default args of BaseModel can be called this way
class Hero(BaseModel, str_max_length=15, extra="forbid"):
    id: int
    name: str = "Thor"

# Testing a name
Hero(id=2, name="Nice Looking Minotausorus")

ValidationError: 1 validation error for Hero
name
  String should have at most 15 characters [type=string_too_long, input_value='Nice Looking Minotausorus', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_long

## pydantic.dataclasses

In [13]:
# WAY 3 : Using the @dataclass decorator from pydantic.dataclasses without BaseModel
from pydantic.dataclasses import dataclass
from datetime import datetime

@dataclass(config=ConfigDict(str_max_length=10, validate_assignement=True))
class Hero:
    id: int
    name: str = "Thor"
    signup_ts : datetime = None

Hero(id=0, name=2)

ValidationError: 1 validation error for Hero
name
  Input should be a valid string [type=string_type, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

# @field_validator



In [ ]:
# DEMO - VALIDATEUR DE CHAMPS
from pydantic import BaseModel, Field, ValidationError, field_validator

class Utilisateur(BaseModel):
    nom: str
    age: int

    # 
    @field_validator("age")
    def verifier_age(cls, age):
        if age < 0:
            raise ValueError("L'âge doit être positif")
        return age

# Utilisation
try:
    utilisateur = Utilisateur(nom="Alice", age=-1)
except ValidationError as e:
    print(e)  # Affiche: 1 validation error for Utilisateur
              # age
              #   L'âge doit être positif (type=value_error)


1 validation error for Utilisateur
age
  Value error, L'âge doit être positif [type=value_error, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


In [1]:
# Protection contre l'injection avec convertissage automatique et si pas possible, renvoie l'erreur
from pydantic import BaseModel, ValidationError

class Utilisateur(BaseModel):
    id: int
    nom: str = 'John Doe'
    age: int

# Utilisation
utilisateur = Utilisateur(id='123', age='30')
print(utilisateur)
utilisateur = Utilisateur(id='123', age='bool')
print(utilisateur)

id=123 nom='John Doe' age=30


ValidationError: 1 validation error for Utilisateur
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bool', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing

In [ ]:
# Sérialisation/Désérialisation
utilisateur = Utilisateur(id='123', age='30')
json_serialize = utilisateur.model_dump_json() # Sérialisation
print(utilisateur.model_dump_json())
Utilisateur.model_validate_json(json_serialize) # Désérialisation

{"id":123,"nom":"John Doe","age":30}


Utilisateur(id=123, nom='John Doe', age=30)

In [ ]:
from typing import Generic, TypeVar

V = TypeVar("V")

class Boite(Generic[V]):
    def __init__(self, contenu: V):
        self.contenu = contenu

    def get_contenu(self) -> V:
        return self.contenu

boite_int = Boite(123)
boite_str = Boite("Hello")

print(boite_int.get_contenu())  # 123
print(boite_str.get_contenu())  # "Hello"


123
Hello


In [ ]:
from pydantic import BaseModel, ConfigDict

class User(BaseModel, extra='allow'):
    id: int
    name: str = "Jane Doe"

    model_config = ConfigDict(str_max_length=15)

In [ ]:
User(id=1, name="Paulo", unknown="Unknown")

User(id=1, name='Paulo', unknown='Unknown')

# Type hints

https://mypy.readthedocs.io/en/latest/cheat_sheet_py3.html

## Python 3.10+ or Typing

In [10]:
def get_name_with_age(name: str, age: int) -> str:
    return name + " is this old: " + str(age)

In [11]:
"""
Depuis Python 3.10, la plupart des types de séquence n'ont pas besoin de typing
"""
def process_a(items: list[str]):
    for item in items:
        print(item)

In [12]:
"""
Tuple can have multiple types declared.

The variable items_t is a tuple with 3 items, an int, another int, and a str.
The variable items_s is a set, and each of its items is of type bytes.
"""

def process_b(items_t: tuple[int, int, str], items_s: set[bytes]):
    return items_t, items_s

In [13]:
"""
Pour les dictionnaires, la 1ere valeur représente la clé et la seconde la valeur
"""

def process_c(prices: dict[str, float]):
    for item_name, item_price in prices.items():
        print(item_name)
        print(item_price)

In [ ]:
"""
Pipe | is used for int or str here. It's the same as using Union from typing
"""
from typing import Union # noqa

def process_d(item: int | str, value: Union[int, str]):
    print(item)
    print(value)

In [16]:
"""
Optional can be declare with python with type | None
Typing has 2 alternatives:
- Optional[type]
- Union[type, None]
"""

from typing import Optional, Union # noqa

def say_hi(name: str | None = None, age: Optional[int] = None, height: Union[int, None] = None):
    if name is not None:
        print(f"Hey {name}!")
    else:
        print("Hello World")

In [17]:
"""
You can use the same builtin types as generics (with square brackets and types inside):

list
tuple
set
dict
"""

'\nYou can use the same builtin types as generics (with square brackets and types inside):\n\nlist\ntuple\nset\ndict\n'

In [18]:
"""
Use for class instances
"""

class Person:
    def __init__(self, name: str):
        self.name = name


def get_person_name(one_person: Person):
    return one_person.name

## Annotated

In [22]:
"""
Python itself doesn't do anything with this Annotated. And for editors and other tools, the type is still str.

But you can use this space in Annotated to provide FastAPI with additional metadata about how you want your application to behave.

The important thing to remember is that the first type parameter you pass to Annotated is the actual type. The rest, is just metadata for other tools.

For now, you just need to know that Annotated exists, and that it's standard Python. 😎

Later you will see how powerful it can be.
"""
from typing import Annotated # noqa


def say_hello(name: Annotated[str, "this is just metadata"]) -> str:
    return f"Hello {name}"